<a href="https://colab.research.google.com/github/Yank224/Proyecto/blob/main/proyecto_taller_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instituto Tecnológico de Costa Rica
### Reconocimiento de Rostros
Author: [Johan Rodríguez](https://github.com/Johanx22x)

Note: When using this notebook, you must select a GPU Hardware Accelerator instead of CPU. Go to Runtime->Change Runtime Type and select any option that uses GPU.

# Dependencies

In [ ]:
!pip install face_recognition         # Install face_recognition package using pip
import face_recognition
import cv2                            # Import OpenCV
import numpy as np
from matplotlib import pyplot as plt  # pyplot

# ipywidgets packages
import ipywidgets as widgets
from IPython.display import display, clear_output

# Live Streaming dependencies
from IPython.display import Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566171 sha256=91cdf572eed8a2cfb6810e053b776a84da84ceee5d1060753ff61792a735e5a8
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


# Utils

## In-memory data

In [ ]:
registered_faces = {}
registered_professor = []
registered_estudent = []
registered_misceláneao = []
registered_aula = []
registered_curso = []

## face recognition

In [ ]:
def check_if_face_registered(face_encoding):
  for _, registered_face in registered_faces.items():
    if True in face_recognition.compare_faces([face_encoding], registered_face):
      return True
  return False

## Real time streaming

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 800, 600);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 800; //video.videoWidth;
      captureCanvas.height = 600; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

In [ ]:
# JavaScript code to stop the webcam stream
js_stop_stream = """
    const stream = video.srcObject;
    const tracks = stream.getTracks();

    tracks.forEach(function(track) {
        track.stop();
    });

    video.srcObject = null;
    div.style.display = 'none';  // Hide the container
"""

# Function to stop the video streaming
def stop_video_stream():
    display(Javascript(js_stop_stream))

# Register Faces

In [ ]:
# Create a text input widget for the name
name_input = widgets.Text(
    placeholder='Enter your name',
    description='Name:',
    disabled=False
)

# Create a file upload widget
upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)

or_text = widgets.HTML(
    value=" or "
)

# Create a button to trigger face registration
register_button = widgets.Button(description='Register Face')

# Create a button to trigger image capture
capture_button = widgets.Button(description='Capture Image')

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

# Initialize variables to store the uploaded image and name
uploaded_image_data = None
registered_name = None

# Define a function to handle image upload
def handle_upload(change):
    global uploaded_image_data
    uploaded_filename = list(upload_button.value.keys())[0]
    uploaded_image_data = upload_button.value[uploaded_filename]['content']

    with output:
        clear_output()
        display(widgets.Image(value=uploaded_image_data))

    # Assuming uploaded_image_data is binary image data in bytes
    image_array = np.frombuffer(uploaded_image_data, np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # Convert the image to RGB format
    uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def handle_capture(button):
  global uploaded_image_data
  # start streaming video from webcam
  video_stream()
  js_reply = video_frame("Capturing...", "")

  stop_video_stream()  # Stop the video stream after capturing the frame

  if not js_reply:
    with output:
      clear_output()
      print("Error capturing image!")

  # convert JS response to OpenCV Image
  uploaded_image_data = js_to_image(js_reply["img"])

  with output:
    clear_output()
    # Encode the captured frame as PNG
    _, buffer = cv2.imencode('.png', uploaded_image_data)

    # Convert the encoded image to bytes
    image_bytes = buffer.tobytes()
    display(widgets.Image(value=image_bytes))

# Define a function to handle face registration
def handle_register(button):
    global registered_name

    if uploaded_image_data is not None:
        registered_name = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
              clear_output()
              print("No face detected in the image!")
              return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
              clear_output()
              print("More than one face detected in the image!")
              return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
              clear_output()
              print("Face already registered!")
              return

        with output:
            clear_output()
            print(f"Face registered for {registered_name}")
            registered_faces[registered_name] = face_encodings[0]

upload_button.observe(handle_upload, names='value')
capture_button.on_click(handle_capture)
register_button.on_click(handle_register)

# Create a container for the widgets

container = widgets.VBox([name_input, widgets.HBox([upload_button, or_text, capture_button]), register_button, output])
def clear_container():
    with output:
        clear_output()
    container.children = [name_input, widgets.HBox([upload_button, or_text, capture_button]), register_button, output]

# Display the container
display(container)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(registered_faces)

{'yan': array([-0.19840746,  0.10490777,  0.002349  , -0.03337994, -0.10707865,
       -0.04887335, -0.01802113, -0.1499896 ,  0.1767804 , -0.08063367,
        0.18253526, -0.07554857, -0.18101546, -0.0497798 ,  0.01124327,
        0.08425128, -0.19936356, -0.11755285, -0.02751706, -0.07361132,
        0.08751081, -0.0164598 ,  0.04445636,  0.08903539, -0.17329852,
       -0.327263  , -0.09928639, -0.12730055,  0.04196533, -0.07026684,
        0.06362744,  0.00687331, -0.17445651, -0.0731204 , -0.03713424,
        0.06931205, -0.07709864, -0.09470432,  0.24010786, -0.09623665,
       -0.18276185, -0.07383379,  0.06703892,  0.24357656,  0.19491312,
        0.02204439, -0.0201293 , -0.07634059,  0.18385834, -0.26275936,
        0.02217393,  0.16866648,  0.00382045,  0.05196533,  0.06301084,
       -0.11599252,  0.09000096,  0.18699077, -0.25750548,  0.02333865,
        0.03427326, -0.07824458,  0.02460614,  0.01478358,  0.20234513,
        0.09531493, -0.08229455, -0.05270286,  0.1637884

In [ ]:
print(registered_name)

Joshua


In [ ]:
from zmq.constants import NULL

def agregar_profesor():
    print("informacion adicional de: ", registered_name)
    cedula = input("Ingrese el id: ")
    nombre = registered_name
    fecha= input("Ingrese su fecha de nacimiento(D/M/A): ")
    nacionalidad= input("Ingrese su nacionalidad ")
    telefono= input("Ingrese su numero de telefono de casa: ")
    celular=input("Ingrese su numero de celular: ")
    telefonos={"casa": telefono,        #diccionario para agregar los dos numeros de telefono
               "celular": celular
               }
    cant = int(input("cuantos correos electronicos desea agregar? ")) #Para crear listas con correos dependiendo de cuantos el usuario quiere ingresar
    imails = []
    while cant >0:
      imail = input("Ingrese su dirección de correo electronico: ")
      imails.append(imail)
      cant-=1
    direccion= input("Ingrese su dirección: ")
    especialidad= input("Ingrese su especialidad: ")
    estado= input("Ingrese su estado civil: ")   #adicional
    asignatura = input("Ingrese la asignatura que enseña: ") #adicional
    imagen=registered_faces

    # Crear un diccionario con la información del profesor
    profesor = {
        "Identificacion": cedula,
        "nombre": nombre,
        "Fecha de nacimiento": fecha,
        "Nacionalidad": nacionalidad,
        "Números de teléfono y celular": telefonos,
        "imail ": imails,
        "Dirección": direccion,
        "Especialidad": especialidad,
        "Estado civil": estado,
        "Asignatura que enseña": asignatura,
        "imagen del profesor": imagen
        }
    registered_professor.append(profesor)
    validador=False

    print("Profesor agregado exitosamente.")
    print(registered_professor)


def agregar_estudiante():
    print("informacion adicional de: ", registered_name)
    cedula = input("Ingrese el id: ")
    nombre = registered_name
    fecha= input("Ingrese su fecha de nacimiento(D/M/A): ")
    nacionalidad= input("Ingrese su nacionalidad ")
    telefono= input("Ingrese su numero de telefono de casa: ")# hacerlo en diccionarios
    celular=input("Ingrese su numero de celular: ")
    telefonos={"casa": telefono,
               "celular": celular
               }
    cant = int(input("cuantos correos electronicos desea agregar? ")) #Para crear listas con correos dependiendo de cuantos el usuario quiere ingresar
    imails = []
    while cant >0:
      imail = input("Ingrese su dirección de correo electronico: ")
      imails.append(imail)
      cant-=1
    direccion= input("Ingrese su dirección: ")
    carrera= input("Ingrese su carrera: ")
    genero= input("Ingrese su genero: ")   #adicional
    estatura= input("Ingrese su estatura: ") #adicional
    imagen=registered_faces



    # Crear un diccionario con la información del estudiante
    estudiante = {
        "Identificacion": cedula,
        "Nombre": nombre,
        "Fecha de nacimiento": fecha,
        "Nacionalidad": nacionalidad,
        "Números de teléfono y celular": telefonos,
        "imail ": imails,
        "Dirección": direccion,
        "carrera": carrera,
        "Genero": genero,
        "Estatura del estudiante": estatura,
        "imagen del estudiante": imagen
        }
    registered_estudent.append(estudiante)
    validador=False

    print("Estudiante agregado exitosamente.")
    print(registered_estudent)

    # Crear un diccionario con la información del miscelaneo
def agregar_miscelaneo():
    print("informacion adicional de: ", registered_name)
    cedula = input("Ingrese el id: ")
    nombre = registered_name
    fecha= input("Ingrese su fecha de nacimiento(D/M/A): ")
    nacionalidad= input("Ingrese su nacionalidad ")
    telefono= input("Ingrese su numero de telefono de casa: ")# hacerlo en diccionarios
    celular=input("Ingrese su numero de celular: ")
    telefonos={"casa": telefono,
               "celular": celular
               }
    cant = int(input("cuantos correos electronicos desea agregar? ")) #Para crear listas con correos dependiendo de cuantos el usuario quiere ingresar
    imails = []
    while cant >0:
      imail = input("Ingrese su dirección de correo electronico: ")
      imails.append(imail)
      cant-=1
    direccion= input("Ingrese su dirección: ")
    turno= input("Ingrese su turno (Diurno/Nocturno): ")
    genero= input("Ingrese su genero: ")   #adicional
    estatura= input("Ingrese su estatura: ")
    imagen=registered_faces

    micelaneo = {
        "Identificacion": cedula,
        "Nombre": nombre,
        "Fecha de nacimiento": fecha,
        "Nacionalidad": nacionalidad,
        "Números de teléfono y celular": telefonos,
        "imail ": imails,
        "Dirección": direccion,
        "Turno": turno,
        "Genero": genero,
        "Estatura del miscelaneo": estatura,
        "imagen del miscelaneo": imagen
        }
    registered_misceláneao.append(micelaneo)
    validador=False

    print("Miscelaneo agregado exitosamente.")
    print(registered_misceláneao)

def agregar_aula():
    print("informacion adicional del aula")
    numero_de_aula = input("Ingrese el número de aula: ")
    miselaneo = input(" ingrese el miselaneoa que le corresponde este aula: ")


    # Crear un diccionario con la información del aula
    aula = {
        "número de aula": numero_de_aula,
        "miselaneo": miselaneo,
        }
    registered_aula.append(aula)
    print("aula agregada exitosamente.")
    print(registered_aula)

def agregar_curso():
    print("informacion adicional del curso")
    codigo = int(input("Ingrese el codigo del curso: "))
    nombre = input(" ingrese el nombre del curso: ")
    periodo = int(input(" ingrese el periodo: "))
    carrera = input(" a que carrera pertenece el curso: ")
    estudiantes = input( "ingrese los estudiantes que estan en el curso: ")# debe ser la misma que ya se agrego
    profesores = input( " ingrese los profesores que dan el curso: ")# debe ser la misma que ya se agrego
    horario = input(" ingrese el horario de lecciones: ")
    aula = input(" ingrese el aula donde se imparte: ")



    # Crear un diccionario con la información del curso
    curso = {
        "codigo": codigo,
        "nombre": nombre,
        "periodo": periodo,
        "carrera" : carrera,
        "estudiantes": estudiantes,
        "profesores" : profesores,
        "horario" : horario,
        "aula": aula
        }
    registered_curso.append(curso)
    print("curso agregado exitosamente.")
    print(registered_curso)

def salir_del_programa():
    print("Saliendo del programa.")
    exit()

menu = {
    "1": agregar_profesor,
    "2": agregar_estudiante,
    "3": agregar_miscelaneo,
    "4": agregar_aula,
    "5": agregar_curso,
    "6": salir_del_programa
}

opcion=0
while True:
    if opcion!=6:
      print(" Ingrese donde quiere agregar a la persona registada anteriormente ")
      print("1. Profesor(a)")
      print("2. Estudiante")
      print("3. Miscelaneo/a")
      print("4. aula")
      print("5. curso")
      print("6. Salir")

      opcion = input("Selecciona una opción: ")
      if opcion=="6":
        break
      if opcion in menu:
          menu[opcion]()  # Llama a la función asociada a la opción seleccionada
          clear_container()
          registered_faces={}
          display(container)
      else:
          print("Opción no válida. Por favor, selecciona una opción válida.")







In [ ]:
print(registered_estudent)

# Recognize Faces

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''

while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
    break

  # convert JS response to OpenCV Image
  frame = js_to_image(js_reply["img"])

  # Get the face locations from the frame
  face_locations = face_recognition.face_locations(frame)

  # For each face in the frame
  for (top, right, bottom, left) in face_locations:
      # Draw a rectangle around the face
      cv2.rectangle(
          frame,
          (left, top),
          (right, bottom),
          (0, 0, 255),
          2,
      )
      # Get the face encoding of the face
      face_encoding = face_recognition.face_encodings(frame, [(top, right, bottom, left)])[0]

      # Compare the face encoding to the faces in the database
      matches = face_recognition.compare_faces(list(registered_faces.values()), face_encoding)

      # If there is a match
      if True in matches:
          # Get the index of the match
          match_index = matches.index(True)

          # Get the name of the match
          match_name = list(registered_faces.keys())[match_index]

          # Draw the name of the match on the frame
          cv2.putText(
              frame,
              match_name,
              (left, top - 10),
              cv2.FONT_HERSHEY_SIMPLEX,
              1,
              (0, 0, 255),
              2,
              cv2.LINE_AA,
          )
      else:
          # Draw unknown on the frame
          cv2.putText(
              frame,
              "Unknown",
              (left, top - 10),
              cv2.FONT_HERSHEY_SIMPLEX,
              1,
              (0, 0, 255),
              2,
              cv2.LINE_AA,
          )

  # Show image
  plt.imshow(frame)
  plt.show()

NameError: ignored